In [1]:
import pandas as pd
import numpy as np
from tmdbv3api import TMDb
import json
import requests
tmdb = TMDb()
with open("api-key.txt", mode = 'r') as f:
    tmdb.api_key = f.readline()

### Getting the data of movies released in 2018 from Wikipedia and TMDb

There are no good consolidated datasets for movies released after 2017 so we are going to get the list of movies from Wikipedia and fetch relevant data for them using the TMDb API.

In [2]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2018"
df1 = pd.read_html(link, header=0)[2]
df2 = pd.read_html(link, header=0)[3]
df3 = pd.read_html(link, header=0)[4]
df4 = pd.read_html(link, header=0)[5]

In [3]:
df1.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]


In [7]:
print(df1.shape, df2.shape, df3.shape, df4.shape)

(73, 6) (63, 6) (66, 6) (66, 6)


In [8]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [11]:
data

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,5,Insidious: The Last Key,Universal Pictures / Blumhouse Productions / S...,Adam Robitel (director); Leigh Whannell (scree...,[2]
1,JANUARY,5,The Strange Ones,Vertical Entertainment,Lauren Wolkstein (director); Christopher Radcl...,[3]
2,JANUARY,5,Stratton,Momentum Pictures,"Simon West (director); Duncan Falconer, Warren...",[4]
3,JANUARY,10,Sweet Country,Samuel Goldwyn Films,"Warwick Thornton (director); David Tranter, St...",[5]
4,JANUARY,12,The Commuter,Lionsgate / StudioCanal / The Picture Company,Jaume Collet-Serra (director); Byron Willinger...,[6]
...,...,...,...,...,...,...
263,DECEMBER,25,Holmes & Watson,Columbia Pictures / Gary Sanchez Productions,Etan Cohen (director/screenplay); Will Ferrell...,[162]
264,DECEMBER,25,Vice,Annapurna Pictures / Plan B Entertainment,Adam McKay (director/screenplay); Christian Ba...,[136]
265,DECEMBER,25,On the Basis of Sex,Focus Features,Mimi Leder (director); Daniel Stiepleman (scre...,[223]
266,DECEMBER,25,Destroyer,Annapurna Pictures,"Karyn Kusama (director); Phil Hay, Matt Manfre...",[256]


#### Using a function for fetch the genres of the movies from the IMDB data

In [12]:
from tmdbv3api import Movie
tmdb_movie = Movie()
def get_genre(title):
    '''Takes in the title of a movie, requests the genres from the TMDb api'''
    genres = []
    result = tmdb_movie.search(title)
    movie_id = result[0].id
    response = requests.get(f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb.api_key}")
    data_json = response.json()
    if data_json['genres']:
        for i in range(0,len(data_json['genres'])):
            genres.append(data_json['genres'][i]['name'])
        return " ".join(genres)
    else:
        np.NaN

In [14]:
print(get_genre(data['Title'][0]))

Mystery Horror Thriller


In [13]:
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

Dropping columns that we don't need

In [15]:
data_2018 = data[['Title','Cast and crew','genres']]

In [16]:
data_2018

,Title,Cast and crew,genres
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller
...,...,...,...
263,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime
264,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure
265,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History
266,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action


**Splitting the cast and crew column into four columns ```director_name, actor_1_name, actor_2_name, actor_3_name``` to match the format of the precious dataset.**

In [17]:
def get_director(x):
    if " (director)" in x:
        return x.split(" (director)")[0]
    elif " (directors)" in x:
        return x.split(" (directors)")[0]
    else:
        return x.split(" (director/screenplay)")[0]

In [18]:
data_2018['director_name'] = data_2018['Cast and crew'].map(lambda x: get_director(x))

<ipython-input-18-f32f153d75e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2018['director_name'] = data_2018['Cast and crew'].map(lambda x: get_director(x))


In [19]:
data_2018

,Title,Cast and crew,genres,director_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller,Adam Robitel
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra
...,...,...,...,...
263,Holmes & Watson,Etan Cohen (director/screenplay); Will Ferrell...,Mystery Adventure Comedy Crime,Etan Cohen
264,Vice,Adam McKay (director/screenplay); Christian Ba...,Thriller Science Fiction Action Adventure,Adam McKay
265,On the Basis of Sex,Mimi Leder (director); Daniel Stiepleman (scre...,Drama History,Mimi Leder
266,Destroyer,"Karyn Kusama (director); Phil Hay, Matt Manfre...",Thriller Crime Drama Action,Karyn Kusama


In [22]:
def get_actor(x, number = 1):
    if number == 2 and len((x.split("screenplay); ")[-1]).split(", ")) < 2:
        return np.NaN
    if number == 3 and len((x.split("screenplay); ")[-1]).split(", ")) < 3:
        return np.NaN
    return ((x.split("screenplay); ")[-1]).split(", ")[number - 1])

In [23]:
data_2018['actor_1_name'] = data_2018['Cast and crew'].map(lambda x: get_actor(x))
data_2018['actor_2_name'] = data_2018['Cast and crew'].map(lambda x: get_actor(x, 2))
data_2018['actor_3_name'] = data_2018['Cast and crew'].map(lambda x: get_actor(x, 3))

<ipython-input-23-1a1f8be3783a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2018['actor_1_name'] = data_2018['Cast and crew'].map(lambda x: get_actor(x))
<ipython-input-23-1a1f8be3783a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2018['actor_2_name'] = data_2018['Cast and crew'].map(lambda x: get_actor(x, 2))


In [25]:
data_2018.head()

,Title,Cast and crew,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Adam Robitel (director); Leigh Whannell (scree...,Mystery Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Lauren Wolkstein (director); Christopher Radcl...,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,"Simon West (director); Duncan Falconer, Warren...",Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,"Warwick Thornton (director); David Tranter, St...",Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Jaume Collet-Serra (director); Byron Willinger...,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


#### Final reformatting and cleaning before consolidating the data.

In [26]:
data_2018 = data_2018.rename(columns={'Title':'movie_title'})

In [28]:
new_2018 = data_2018.drop("Cast and crew", axis = 1)

In [29]:
new_2018.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name
0,Insidious: The Last Key,Mystery Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell
1,The Strange Ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus
2,Stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan
3,Sweet Country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,NaN
4,The Commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson


In [30]:
new_2018.isnull().sum()

movie_title       0
genres            3
director_name     0
actor_1_name      0
actor_2_name      9
actor_3_name     26
dtype: int64

In [31]:
new_2018['actor_2_name'] = new_2018['actor_2_name'].replace(np.nan, 'unknown')
new_2018['actor_3_name'] = new_2018['actor_3_name'].replace(np.nan, 'unknown')

In [ ]:
new_2018['genres'] = new_2018['genres'].replace(np.nan, 'unknown')

In [32]:
new_2018['movie_title'] = new_2018["movie_title"].str.lower()

In [33]:
new_2018['comb'] = new_2018['actor_1_name'] + ' ' + new_2018['actor_2_name'] + ' '+ new_2018['actor_3_name'] + ' '+ new_2018['director_name'] +' ' + new_2018['genres']

In [34]:
new_2018.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,insidious: the last key,Mystery Horror Thriller,Adam Robitel,Lin Shaye,Angus Sampson,Leigh Whannell,Lin Shaye Angus Sampson Leigh Whannell Adam Ro...
1,the strange ones,Thriller Drama,Lauren Wolkstein,Alex Pettyfer,James Freedson-Jackson,Emily Althaus,Alex Pettyfer James Freedson-Jackson Emily Alt...
2,stratton,Action Thriller,Simon West,Dominic Cooper,Austin Stowell,Gemma Chan,Dominic Cooper Austin Stowell Gemma Chan Simon...
3,sweet country,Drama History Western,Warwick Thornton,Bryan Brown,Sam Neill,unknown,Bryan Brown Sam Neill unknown Warwick Thornton...
4,the commuter,Action Thriller,Jaume Collet-Serra,Liam Neeson,Vera Farmiga,Patrick Wilson,Liam Neeson Vera Farmiga Patrick Wilson Jaume ...


### Getting the data of movies released in 2019 from Wikipedia and TMDb

Repeating the same process but for 2019 movies

In [72]:
link = "https://en.wikipedia.org/wiki/List_of_American_films_of_2019"
df1 = pd.read_html(link, header=0)[3]
df2 = pd.read_html(link, header=0)[4]
df3 = pd.read_html(link, header=0)[5]
df4 = pd.read_html(link, header=0)[6]

In [73]:
data = df1.append(df2.append(df3.append(df4,ignore_index=True),ignore_index=True),ignore_index=True)

In [74]:
data.head()

,Opening,Opening.1,Title,Production company,Cast and crew,Ref.
0,JANUARY,4,Escape Room,Columbia Pictures,"Adam Robitel (director); Bragi F. Schut, Maria...",[2]
1,JANUARY,4,Rust Creek,IFC Films,Jen McGowan (director); Julie Lipson (screenpl...,[3]
2,JANUARY,4,American Hangman,Hangman Justice Productions,Wilson Coneybeare (director/screenplay); Donal...,[4]
3,JANUARY,11,A Dog's Way Home,Columbia Pictures,Charles Martin Smith (director); W. Bruce Came...,[5]
4,JANUARY,11,The Upside,STX Entertainment,Neil Burger (director); Jon Hartmere (screenpl...,[6]


In [75]:
data['genres'] = data['Title'].map(lambda x: get_genre(str(x)))

In [76]:
data_2019 = data[['Title','Cast and crew','genres']]

**Splitting the cast and crew column into four columns ```director_name, actor_1_name, actor_2_name, actor_3_name``` to match the format of the precious dataset.**

In [77]:
data_2019['director_name'] = data_2019['Cast and crew'].map(lambda x: get_director(str(x)))

<ipython-input-77-7e1af3bf5cb9>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2019['director_name'] = data_2019['Cast and crew'].map(lambda x: get_director(str(x)))


In [78]:
data_2019['actor_1_name'] = data_2019['Cast and crew'].map(lambda x: get_actor(x))
data_2019['actor_2_name'] = data_2019['Cast and crew'].map(lambda x: get_actor(x, 2))
data_2019['actor_3_name'] = data_2019['Cast and crew'].map(lambda x: get_actor(x, 3))

<ipython-input-78-9656e4b4896b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2019['actor_1_name'] = data_2019['Cast and crew'].map(lambda x: get_actor(x))
<ipython-input-78-9656e4b4896b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_2019['actor_2_name'] = data_2019['Cast and crew'].map(lambda x: get_actor(x, 2))


#### Final reformatting and cleaning before consolidating the data.

In [79]:
data_2019 = data_2019.rename(columns={'Title':'movie_title'})

In [80]:
new_2019 = data_2019.drop("Cast and crew", axis = 1)

In [81]:
new_2019.isnull().sum()

movie_title       0
genres            2
director_name     0
actor_1_name      0
actor_2_name      1
actor_3_name     16
dtype: int64

In [82]:
new_2019['actor_2_name'] = new_2019['actor_2_name'].replace(np.nan, 'unknown')
new_2019['actor_3_name'] = new_2019['actor_3_name'].replace(np.nan, 'unknown')

In [83]:
new_2019.isnull().sum()

movie_title      0
genres           2
director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
dtype: int64

In [84]:
new_2019.dropna(how='any',inplace=True)

In [85]:
new_2019['movie_title'] = new_2019['movie_title'].str.lower()

In [86]:
new_2019['comb'] = new_2019['actor_1_name'] + ' ' + new_2019['actor_2_name'] + ' '+ new_2019['actor_3_name'] + ' '+ new_2019['director_name'] +' ' + new_2019['genres']

In [88]:
new_2019.head()

,movie_title,genres,director_name,actor_1_name,actor_2_name,actor_3_name,comb
0,escape room,Thriller Action Mystery Adventure Horror,Adam Robitel,Taylor Russell,Logan Miller,Deborah Ann Woll,Taylor Russell Logan Miller Deborah Ann Woll A...
1,rust creek,Thriller Drama,Jen McGowan,Hermione Corfield,Jay Paulson,Sean O'Bryan,Hermione Corfield Jay Paulson Sean O'Bryan Jen...
2,american hangman,Thriller,Wilson Coneybeare,Donald Sutherland,Vincent Kartheiser,Oliver Dennis,Donald Sutherland Vincent Kartheiser Oliver De...
3,a dog's way home,Drama Adventure Family,Charles Martin Smith,Bryce Dallas Howard,Edward James Olmos,Alexandra Shipp,Bryce Dallas Howard Edward James Olmos Alexand...
4,the upside,Comedy Drama,Neil Burger,Bryan Cranston,Kevin Hart,Nicole Kidman,Bryan Cranston Kevin Hart Nicole Kidman Neil B...


In [89]:
combined_df = new_2018.append(new_2019,ignore_index=True)

In [90]:
combined_df.shape

(507, 7)

In [91]:
old_df = pd.read_csv('new_data.csv')

In [92]:
final_df = old_df.append(combined_df,ignore_index=True)

In [93]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           3
movie_title      0
comb             3
dtype: int64

In [94]:
final_df = final_df.dropna(how='any')

In [95]:
final_df.isna().sum()

director_name    0
actor_1_name     0
actor_2_name     0
actor_3_name     0
genres           0
movie_title      0
comb             0
dtype: int64

In [96]:
final_df.to_csv('pre-2020_data.csv',index=False)